In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import datetime
import re
import cx_Oracle
import numpy as np
# Database connection parameters
db_params = {
    "host": "gc-ue4-psql-cspo-dev01.nonprod.gcp.cswg.com",
    "database": "CSPODB",
    "port": 5432,
    "user": "cspoms",
    "password": "cspoms",
    "sslmode": "verify-ca",
    "sslcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-cert.pem",
    "sslkey": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\client-key.pem",
    "sslrootcert": "D:\\W\\RELEX\\RELEX\\CSPOMS_DEV_Conn\\server-ca.pem"
}

post_conn = psycopg2.connect(**db_params)
post_cur = post_conn.cursor()

In [2]:
import pandas as pd

post_cur.execute("""SELECT *
FROM XXPO_PULLBAK_PARAM
WHERE LPAD(CAST(to_fac_code AS VARCHAR), 3, '0') IN (
    SELECT LPAD(CAST(facility_code AS VARCHAR), 3, '0')
    FROM cspoms_replenishment_config
    WHERE POMS_SOURCE = 'CSPOMS'
    AND STATUS = 'Y'
)""")
data = post_cur.fetchall()

# Get column names from the cursor description
column_names = [desc[0] for desc in post_cur.description]
print(column_names)
# Create DataFrame with data and column names
df = pd.DataFrame(data, columns=column_names)
df = df[['from_fac_code','from_warehouse_num','to_fac_code','incr','vendor_number','max_pallets_allowed','max_weight_allowed','max_cube_allowed','lead_time','op_days']]
# Convert columns to integer type
# columns_to_convert = ['from_fac_code','from_warehouse_num','to_fac_code']
# # Replace or drop non-finite values
# df[columns_to_convert] = df[columns_to_convert].replace([np.inf, -np.inf], np.nan).fillna(0)  # Replace inf with NaN and fill NaN with 0

# Convert columns to integer type
# df[columns_to_convert] = df[columns_to_convert].astype(int)
# Dictionary with new column names
new_column_names = {'from_fac_code': "T2_TABLE_26_FR_FAC",
                    'from_warehouse_num': "T2_TABLE_26_FR_WH",
                    'to_fac_code': "T2_TABLE_26_TO_FAC",
                    'incr': "T2_TABLE_26_ORD_INCR",
                    'vendor_number': "T2_TABLE_26_VENDOR",
                    'max_pallets_allowed': "T2_TABLE_26_MIN_PALLET",
                    'max_weight_allowed': "T2_TABLE_26_MIN_WGHT",
                    'max_cube_allowed': "T2_TABLE_26_MIN_CUBE",
                    'lead_time': "T2_TABLE_26_LEAD_TIME",
                    'op_days': "T2_TABLE_26_OP_DAYS"}
# Original column names with their specified lengths
original_column_names = {
    "FROM_FAC_CODE": 13,
    "FROM_WAREHOUSE_NUM": 18,
    "TO_FAC_CODE": 13,
    "INCR": 8,
    "VENDOR_NUMBER": 14,
    "MAX_PALLETS_ALLOWED": 20,
    "MAX_WEIGHT_ALLOWED": 18,
    "MAX_CUBE_ALLOWED": 16,
    "LEAD_TIME": 10,
    "OP_DAYS": 10
}

# Adjust column names to the specified lengths
adjusted_column_names = {col: col.ljust(length) for col, length in original_column_names.items()}

# Use the rename method to rename columns
df.rename(columns=adjusted_column_names, inplace=True)

# Display the DataFrame with adjusted column names
print(df)
# Use the rename method to rename columns
df.rename(columns=new_column_names, inplace=True)

# Assuming df is your DataFrame
# df = pd.concat([df.iloc[:, :10], df.iloc[:, :10]], axis=1)

# Printing the first few rows of the duplicated DataFrame
print(df.head())
# Reorder columns to have first 3 columns at the beginning
# Copy the first three columns to the beginning of the DataFrame
# df = pd.concat([df[['T2_TABLE_26_FR_FAC', 'T2_TABLE_26_FR_WH', 'T2_TABLE_26_TO_FAC']], df], axis=1)
# Assuming df is your DataFrame
# df = df.astype(str)

# df['LAST_UPDATED_BY'] = -999
# # Assuming df is your DataFrame
# df = pd.concat([df.iloc[:, :11], df.iloc[:, :11]], axis=1)

# # Printing the first few rows of the duplicated DataFrame
# print(df.head())
df

['seq_num', 'from_warehouse_num', 'from_fac_code', 'from_fac_name', 'to_fac_code', 'to_fac_name', 'incr', 'vendor_number', 'max_pallets_allowed', 'max_weight_allowed', 'max_cube_allowed', 'lead_time', 'op_days', 'creation_date', 'created_by', 'last_updated_date', 'last_updated_by', 'request_id']
  from_fac_code from_warehouse_num to_fac_code incr vendor_number  \
0            15                  2          23    P        777965   
1            15                  2          43    P        777965   

  max_pallets_allowed max_weight_allowed max_cube_allowed lead_time op_days  
0                  60              42000             1500         1       3  
1                  60              42000             1500         1       3  
  T2_TABLE_26_FR_FAC T2_TABLE_26_FR_WH T2_TABLE_26_TO_FAC  \
0                 15                 2                 23   
1                 15                 2                 43   

  T2_TABLE_26_ORD_INCR T2_TABLE_26_VENDOR T2_TABLE_26_MIN_PALLET  \
0        

,T2_TABLE_26_FR_FAC,T2_TABLE_26_FR_WH,T2_TABLE_26_TO_FAC,T2_TABLE_26_ORD_INCR,T2_TABLE_26_VENDOR,T2_TABLE_26_MIN_PALLET,T2_TABLE_26_MIN_WGHT,T2_TABLE_26_MIN_CUBE,T2_TABLE_26_LEAD_TIME,T2_TABLE_26_OP_DAYS
0,15,2,23,P,777965,60,42000,1500,1,3
1,15,2,43,P,777965,60,42000,1500,1,3


In [5]:
import pytz
# request_id = self.getRequestIdFromSeq(connection=post_conn, sequence_name="cspoms.cspoms_request_id_s")
request_id = 777
table_main = 'XXPO_PULLBACK_BICEPS2EXA_DATA'
# utc_now = datetime.datetime.now(datetime.timezone.utc)
# Convert UTC time to EST
# est = pytz.timezone('America/New_York')
# est_time = utc_now.astimezone(est).strftime("%Y-%m-%d %H:%M:%S")
# df['created_by'] = "CSPOMS"
# df['creation_date'] = est_time
df['last_updated_by'] = -999
# df['last_update_date'] = est_time
df['request_id'] = request_id
df['process_status'] = "N"
# df['INBOUND_SOURCE'] = 'Oracle'
table_name = "P2P_DATA.XXPO_PULLBACK_BICEPS2EXA_STG"
fields = [col for col in df.columns]
str_sub = ', '.join(['%s'] * len(fields))
# Construct the INSERT query with placeholders for each column
insert_query = f"INSERT INTO {table_name} ({', '.join(fields)}) VALUES ({', '.join([':' + str(i+1) for i in range(len(fields))])})"
print("query_stg::", insert_query)
# Convert the DataFrame to a list of tuples for bulk insertion
data_tuples = [tuple(row) for row in df.itertuples(index=False, name=None)]
print(data_tuples)


# Execute the INSERT statement with the data
oracle_cur.executemany(insert_query, data_tuples)
count = oracle_cur.rowcount
oracle_con.commit()
print(f'INSERTED {count} EXCEL RECORDS TO {table_name} TABLE')

merge_query = f"""
select * from XXPO_PULLBACK_BICEPS2EXA_DATA where RECORD_ID IS NULL;"""
print(merge_query)
oracle_cur.execute(merge_query)
count = oracle_cur.rowcount
print(f'MERGED {count} EXCEL RECORDS TO {table_main} TABLE')

oracle_con.commit()

query_stg:: INSERT INTO P2P_DATA.XXPO_PULLBACK_BICEPS2EXA_STG (T2_TABLE_26_FR_FAC, T2_TABLE_26_FR_WH, T2_TABLE_26_TO_FAC, T2_TABLE_26_ORD_INCR, T2_TABLE_26_VENDOR, T2_TABLE_26_MIN_PALLET, T2_TABLE_26_MIN_WGHT, T2_TABLE_26_MIN_CUBE, T2_TABLE_26_LEAD_TIME, T2_TABLE_26_OP_DAYS, last_updated_by, request_id, process_status) VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13)
[('15', '2', '23', 'P', '777965', '60', '42000', '1500', '1', '3', -999, 777, 'N'), ('15', '2', '43', 'P', '777965', '60', '42000', '1500', '1', '3', -999, 777, 'N')]
INSERTED 2 EXCEL RECORDS TO P2P_DATA.XXPO_PULLBACK_BICEPS2EXA_STG TABLE

MERGE INTO XXPO_PULLBACK_BICEPS2EXA_DATA main
USING (SELECT distinct * FROM XXPO_PULLBACK_BICEPS2EXA_STG WHERE LAST_UPDATED_BY = -999) stg
ON (main.T2_TABLE_26_FR_FAC = stg.T2_TABLE_26_FR_FAC
    AND main.T2_TABLE_26_FR_WH = stg.T2_TABLE_26_FR_WH
    AND main.T2_TABLE_26_TO_FAC = stg.T2_TABLE_26_TO_FAC)
WHEN MATCHED THEN
    UPDATE SET 
        main.T2_TABLE_26_ORD_INCR = 

In [3]:
# Broker_vendor.py
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle

username = 'ebs_apps'
password = 'ebs_apps'
host = 'csdwddb.cswg.com' #EBSD2
port = '1521'
service_name = 'DWD_USR' #EBSD2

# Create a connection
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
oracle_con = cx_Oracle.connect(username, password, dsn)
oracle_cur = oracle_con.cursor()

In [20]:
# Construct the INSERT query with placeholders for each column
insert_query = f"INSERT INTO {table_name} ({', '.join(fields)}) VALUES ({', '.join([':' + str(i+1) for i in range(len(fields))])})"

# # Remove the duplicate column 'last_updated_by'
# fields.remove('last_updated_by')

# Convert the DataFrame to a list of tuples for bulk insertion
data_tuples = [tuple(row) for row in df[fields].itertuples(index=False, name=None)]
print(insert_query)
print(data_tuples)
# Execute the INSERT statement with the data
oracle_cur.executemany(insert_query, data_tuples)

# Commit the transaction
connection.commit()

count = len(data_tuples)
print(f'INSERTED {count} EXCEL RECORDS TO {table_name} TABLE')


INSERT INTO P2P_DATA.XXPO_PULLBACK_BICEPS2EXA_STG (T2_TABLE_26_FR_FAC, T2_TABLE_26_FR_WH, T2_TABLE_26_TO_FAC, T2_TABLE_26_ORD_INCR, T2_TABLE_26_VENDOR, T2_TABLE_26_MIN_PALLET, T2_TABLE_26_MIN_WGHT, T2_TABLE_26_MIN_CUBE, T2_TABLE_26_LEAD_TIME, T2_TABLE_26_OP_DAYS, last_updated_by, last_update_date, request_id, process_status) VALUES (:1, :2, :3, :4, :5, :6, :7, :8, :9, :10, :11, :12, :13, :14)
[(15, 2, 23, 'P', '777965', 60, '42000', '1500', '1', '3', 'CSPOMS', '2024-03-26 08:03:40', 777, 'N'), (15, 2, 43, 'P', '777965', 60, '42000', '1500', '1', '3', 'CSPOMS', '2024-03-26 08:03:40', 777, 'N')]


DatabaseError: ORA-01722: invalid number

In [18]:
# Construct the INSERT query with placeholders for each column
insert_query = f"INSERT INTO {table_name} ({', '.join(fields)}) VALUES ({str_sub})"

# Convert the DataFrame to a list of tuples for bulk insertion
data_tuples = [tuple(row) for row in df[fields].itertuples(index=False, name=None)]

# Execute the INSERT statement with the data
oracle_cur.executemany(insert_query, data_tuples)
count = oracle_cur.rowcount
print(f'INSERTED {count} EXCEL RECORDS TO {table_name} TABLE')


DatabaseError: ORA-01036: illegal variable name/number

In [11]:

# Broker_vendor.py
import pandas as pd
import xml.etree.ElementTree as ET
import psycopg2
from datetime import datetime
import re
import cx_Oracle

username = 'ebs_apps'
password = 'ebs_apps'
host = 'csdwddb.cswg.com' #EBSD2
port = '1521'
service_name = 'DWD_USR' #EBSD2

# Create a connection
dsn = cx_Oracle.makedsn(host, port, service_name=service_name)
oracle_con = cx_Oracle.connect(username, password, dsn)
oracle_cur = oracle_con.cursor()

# dfEXA = oracle_cur.execute('select * from XXPO_PULLBACK_BICEPS2EXA_STG')
# dfEXA.fetchall()

# Execute the merge statement
merge_sql = """
select * from XXPO_PULLBACK_BICEPS2EXA_DATA
"""


# Convert DataFrame to list of tuples
data = [tuple(row) for row in df.values]

# Execute the merge statement for each row of data
for row in data:
    # Print the values in the row tuple
    print("Row tuple:", row)
    oracle_cur.execute(merge_sql,row)
    


# Commit the transaction and close the cursor and connection
oracle_con.commit()
oracle_cur.close()
oracle_con.close()


Row tuple: (15, 2, 23, 'P', '777965', 60, '42000', '1500', '1', '3', -999)


DatabaseError: ORA-01036: illegal variable name/number

In [ ]:
import pandas as pd

# Assuming df1 and df2 are your Pandas DataFrames for PostgreSQL and Oracle, respectively

# Select only the relevant columns for comparison
df1_subset = df1[['T2_TABLE_26_FR_FAC', 'T2_TABLE_26_FR_WH', 'T2_TABLE_26_TO_FAC']]
df2_subset = df2[['T2_TABLE_26_FR_FAC', 'T2_TABLE_26_FR_WH', 'T2_TABLE_26_TO_FAC']]

# Merge the two DataFrames on the common columns for comparison
merged_df = pd.merge(df1_subset, df2_subset, how='left', indicator=True, on=['T2_TABLE_26_FR_FAC', 'T2_TABLE_26_FR_WH', 'T2_TABLE_26_TO_FAC'])

# Identify records that are in df1 but not in df2
to_insert = merged_df[merged_df['_merge'] == 'left_only'][['T2_TABLE_26_FR_FAC', 'T2_TABLE_26_FR_WH', 'T2_TABLE_26_TO_FAC']]

# Perform updates for existing records
# Assuming update_df is a DataFrame containing updates for existing records

# Assuming you have a function to perform updates and inserts
def update_record(row):
    # Implement your logic to update or insert records
    pass

# Apply the update_record function to each row in the DataFrame
to_insert.apply(update_record, axis=1)

# After performing updates and inserts, you may need to commit changes to your database
# Assuming you have a function to commit changes
def commit_changes():
    # Implement your logic to commit changes to the database
    pass

# Commit changes to the database
commit_changes()
